In [2]:
# Import the libraries we will be using
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.model_selection import cross_val_score
from collections import defaultdict

import sys
sys.path.append("..")

sns.set(font_scale=1.5)
pd.set_option('display.max_rows', 50)

In [3]:
# Import the ratings dataset
df = pd.read_csv('./scotch.csv')
df.head()

,NAME,wyne,yellow,v.pale,pale,p.gold,gold,o.gold,f.gold,bronze,...,smoke.1,sweet.1,spice.1,oil.1,salt.1,arome.1,ling,long,very,quick
0,Aberfeldy,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Aberlour,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Ardberg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Ardmore,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Auchentoshan,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [4]:
X= df.iloc[:,1:]
X.head(5)

,wyne,yellow,v.pale,pale,p.gold,gold,o.gold,f.gold,bronze,p.amber,...,smoke.1,sweet.1,spice.1,oil.1,salt.1,arome.1,ling,long,very,quick
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
y = df.iloc[:,0:1]
y.head(5)

,NAME
0,Aberfeldy
1,Aberlour
2,Ardberg
3,Ardmore
4,Auchentoshan


In [6]:
df.dtypes

NAME       object
wyne        int64
yellow      int64
v.pale      int64
pale        int64
            ...  
arome.1     int64
ling        int64
long        int64
very        int64
quick       int64
Length: 69, dtype: object

In [7]:
#Here is a function that will compute the distance (between users) using as many metrics as you want:
distance_measure = 'euclidean'
user = X.iloc[[19],]
#user = X.iloc[:1,]

user.head(1)
#type(distance_measure)
#distance.pdist(X, distance_measure)
def distances(y, dist_1, dist_2):
    y_new = pd.DataFrame()
    y_new = y
    euclid = distance.cdist(X, user, dist_1)
    euclid_df = pd.DataFrame(euclid, columns = ['Euclidean'])
    jaccard = distance.cdist(X, user, dist_2)
    jaccard_df = pd.DataFrame(jaccard, columns = ['Jaccard'])
    y_new.insert(1, "Euclidean", euclid_df)
    y_new.insert(2, "Jaccard", jaccard_df)
    return y_new
    
distances(y, 'euclidean', 'jaccard').head(15)



,NAME,Euclidean,Jaccard
0,Aberfeldy,4.242641,0.900000
1,Aberlour,4.358899,0.863636
2,Ardberg,3.162278,0.666667
3,Ardmore,4.123106,0.944444
4,Auchentoshan,3.741657,0.823529
5,Aultmore,3.464102,0.800000
6,Balblair,3.464102,0.750000
7,Balmenach,4.123106,0.850000
8,Balvenie,3.872983,0.882353
9,Banff,3.605551,0.812500


In [8]:
y_sorted = y.sort_values(by='Jaccard', ascending=True)
y_sorted.head(50)

,NAME,Euclidean,Jaccard
19,Bunnahabhain,0.000000,0.000000
48,Glenglassaugh,3.000000,0.642857
108,Tullibardine,3.316625,0.647059
57,Glenmorangie,3.464102,0.666667
18,Bruichladdich,3.162278,0.666667
2,Ardberg,3.162278,0.666667
40,Glen Deveron,3.605551,0.684211
26,Cragganmore,3.464102,0.705882
43,Glen Elgin,3.464102,0.705882
89,Oban,3.872983,0.714286
